In [1]:
import pymongo
from pymongo import MongoClient, UpdateOne
import json

# Adding for query to find top ten stations 
#from pymongo.collection import Collection
#from pymongo.aggregation import Aggregation

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# check our list of databases
print(mongo.list_database_names())

['admin', 'autosaurus', 'chicago_bikes', 'classDB', 'config', 'epa', 'fruits_db', 'gardenDB', 'local', 'met', 'travel_db', 'uk_food']


In [4]:
# assign the database to a variable name
db = mongo.chicago_bikes

In [5]:
# review the collections in our new database
print(db.list_collection_names())

['weather_daily', 'Top10Routes', 'Top10EndStations', 'withStationName', 'withoutStationName', 'Top10StartStations', 'divvy_ridedata', 'divvy_ridedata_merged']


In [ ]:
pipeline = [
    {
        "$match": {
            "$and": [
                { "start_lat": {"$or": [{ "$ne": "" }, {"$ne": 0}]}},
                { "start_lng": {"$or": [{ "$ne": "" }, {"$ne": 0}]}},
                { "end_lat": {"$or": [{ "$ne": "" }, {"$ne": 0}]}},
                { "end_lng": {"$or": [{ "$ne": "" }, {"$ne": 0}]}}
            ]
        }
    },
    {
        "$addFields": {
            "start_lat": { "$toDouble": "$start_lat" },
            "start_lng": { "$toDouble": "$start_lng" },
            "end_lat": { "$toDouble": "$end_lat" },
            "end_lng": { "$toDouble": "$end_lng" }
        }
    },
    {
        "$addFields": {
            "distance": {
                "$sqrt": {
                    "$add": [
                        {
                            "$pow": [
                                { "$subtract": ["$end_lat", "$start_lat"] },
                                2
                            ]
                        },
                        {
                            "$pow": [
                                {
                                    "$multiply": [
                                        { "$subtract": ["$end_lng", "$start_lng"] },
                                        { "$cos": { "$avg": ["$start_lat", "$end_lat"] } }
                                    ]
                                },
                                2
                            ]
                        }
                    ]
                }
            }
        }
    },
    {
        "$sort": {"distance": -1}
    }
    {"$out": "RouteDistance"}
]

# Perform the aggregation
result = list(withLatLong.aggregate(pipeline))

# Assign to a variable
RouteDistance = db['RouteDistance']

In [ ]:
# review the collections in our new database
print(db.list_collection_names())